In [ ]:
import pandas as pd
import os
import shutil
import keras

from keras.applications.resnet_v2 import preprocess_input as resnet_preprocessor
from keras.applications.densenet import preprocess_input as densenet_preprocessor

In [ ]:
data_src = '/content/drive/MyDrive/Project/project_data'
!unzip -qq drive/My\Drive/Project/project_data/Post-hurricane.zip

In [ ]:
data_dir = '/content/Hurricane Data'

os.mkdir(data_dir)
shutil.move('/content/train_another', data_dir)
shutil.move('/content/validation_another', data_dir)
shutil.move('/content/test', data_dir)
shutil.move('/content/test_another', data_dir)

'/content/Hurricane Data/test_another'

In [ ]:
# Defining the path where our data is currently loaded

data_dir_dict = {}

data_dir_dict['train_dir'] = os.path.join(data_dir,'train_another')
data_dir_dict['val_dir'] = os.path.join(data_dir,'validation_another')
data_dir_dict['balanced_test_dir'] = os.path.join(data_dir, 'test')
data_dir_dict['imbalanced_test_dir'] = os.path.join(data_dir, 'test_another')

print('Data is Mounted on the Local Runtime in the following Directories :- \n')
data_dir_dict

Data is Mounted on the Local Runtime in the following Directories :- 



{'train_dir': '/content/Hurricane Data/train_another',
 'val_dir': '/content/Hurricane Data/validation_another',
 'balanced_test_dir': '/content/Hurricane Data/test',
 'imbalanced_test_dir': '/content/Hurricane Data/test_another'}

In [ ]:
from data_preprocessing import Simple_Data_Generator

from data_preprocessing import SotA_Data_Generator

from model_evaluation import model_performance

from model_evaluation import plot_auc_roc

from model_evaluation import plot_confusion_matrix

from model_evaluation import evaluation_metrics

In [ ]:
train_gen, val_gen, bal_test_gen, imb_test_gen = Simple_Data_Generator(data_dir_dict,
                                                                       image_dim = (128,128),
                                                                       batch_size = 64)

Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 9000 files belonging to 2 classes.


In [ ]:
from models import Baseline_ANN

ANN = Baseline_ANN()

In [ ]:
historyANN = ANN.fit(train_gen,
                     epochs = 25,
                     validation_data = val_gen)

Epoch 1/25
157/157 [==============================] - 10s 40ms/step - loss: 0.8582 - accuracy: 0.6814 - val_loss: 1.3602 - val_accuracy: 0.5410
Epoch 2/25
157/157 [==============================] - 6s 38ms/step - loss: 0.5076 - accuracy: 0.7645 - val_loss: 0.5383 - val_accuracy: 0.7625
Epoch 3/25
139/157 [=========================>....] - ETA: 0s - loss: 0.4602 - accuracy: 0.7893

KeyboardInterrupt: ignored

In [ ]:
ANN.summary()

In [ ]:
model_performance(historyANN)

In [ ]:
plot_auc_roc(ANN, bal_test_gen, imb_test_gen)

In [ ]:
plot_confusion_matrix(ANN, bal_test_gen, imb_test_gen)

In [ ]:
ANN_preformance = evaluation_metrics(ANN, bal_test_gen, imb_test_gen)
ANN_preformance

In [ ]:
from models import Regularized_CNN
CNN = Regularized_CNN(input_shape = (128,128,3))

In [ ]:
historyCNN = CNN.fit(train_gen,
                     epochs = 25,
                     validation_data = val_gen)

In [ ]:
model_performance(historyCNN)

In [ ]:
plot_auc_roc(CNN, bal_test_gen, imb_test_gen)

In [ ]:
plot_confusion_matrix(CNN, bal_test_gen, imb_test_gen)

In [ ]:
CNN_performance = evaluation_metrics(CNN, bal_test_gen, imb_test_gen)
CNN_performance

In [ ]:
resnet_train_gen, resnet_val_gen, resnet_bal_test_gen, resnet_imb_test_gen = SotA_Data_Generator(data_dir_dict,
                                                                                                 resnet_preprocessor,
                                                                                                 target_dim = (224,224),
                                                                                                 batch_size = 64)

In [ ]:
from models import ResNet50_V2

resnet = ResNet50_V2(input_shape = (128, 128, 3))

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
def step_size(data):
  step = (data.n)//(data.batch_size)
  return step

early_stopping = EarlyStopping(monitor='val_accuracy',
                              patience=5,
                              verbose=0,
                              restore_best_weights=True)

In [ ]:
historyResNet = resnet.fit(resnet_train_gen,
                           epochs=25,
                           validation_data = resnet_val_gen,
                           steps_per_epoch = step_size(resnet_train_gen),
                           validation_steps = step_size(resnet_val_gen),
                           callbacks=[early_stopping])

In [ ]:
model_performance(historyResNet)

In [ ]:
plot_auc_roc(resnet, resnet_bal_test_gen, resnet_imb_test_gen, preprocessor = True)

In [ ]:
plot_confusion_matrix(resnet, resnet_bal_test_gen, resnet_imb_test_gen, preprocessor = True)

In [ ]:
resnet_performance = evaluation_metrics(resnet, resnet_bal_test_gen, resnet_imb_test_gen, preprocessor = True)
resnet_performance

In [ ]:
densenet_train_gen, densenet_val_gen, densenet_bal_test_gen, densenet_imb_test_gen = SotA_Data_Generator(data_dir_dict,
                                                                                                 densenet_preprocessor,
                                                                                                 target_dim = (224,224),
                                                                                                 batch_size = 64)

In [ ]:
from models import DenseNet_201
densenet = DenseNet_201(input_shape = (128, 128, 3))

In [ ]:
historyDenseNet = densenet.fit(densenet_train_gen,
                           epochs=1,
                           validation_data = densenet_val_gen,
                           steps_per_epoch = step_size(densenet_train_gen),
                           validation_steps = step_size(densenet_val_gen),
                           callbacks=[early_stopping])

In [ ]:
model_performance(historyDenseNet)

In [ ]:
plot_auc_roc(densenet, densenet_bal_test_gen, densenet_imb_test_gen,preprocessor = True)

In [ ]:
plot_confusion_matrix(densenet, densenet_bal_test_gen, densenet_imb_test_gen,preprocessor = True)

In [ ]:
densenet_performance = evaluation_metrics(densenet, densenet_bal_test_gen, densenet_imb_test_gen,preprocessor = True)
densenet_performance

In [ ]:
ANN_preformance

In [ ]:
bal_models_performance = {'Models': ['Baseline ANN', 'Regularized CNN', 'Resnet50 V2', 'DenseNet201'],
                          'Accuracy': [ANN_preformance['Balanced Dataset'].to_list(), CNN_preformance['Balanced Dataset'].to_list(),
                                       resnet_preformance['Balanced Dataset'].to_list(), densenet_preformance['Balanced Dataset'].to_list()]}

In [ ]:
ANN_preformance['Balanced Dataset'].to_list()

In [ ]:
5ANN_preformance[['Metrics', 'Balanced Dataset']].set_index('Metrics').T.

In [ ]:
CNN_performance

In [ ]:
densenet_performance

In [ ]:
resnet_performance

In [ ]:
import time

In [ ]:
start = time.time()

945+45*43534534

end = time.time()

exc = end-start

In [ ]:
exc

0.00010800361633300781